In [1]:
# Experiment 1 - With two output layers and 2 different loss function for leakageness and coordinates. Unable to do hyperparameter tuning since so far i couldnt figure
# out how to do hyper parameter tuning on functional api
# Hence i tried to implement sequential api with 6 outputs and 2 different loss function and do hyper parameter tuning. However i am not able to give 2 loss function 
# in that case. This experiment of giving two different loss function for each output neuron of neural network is being given in custom_loss.ipynb. 

# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
# Experiment 3 - Use the idea of masking for the case where when we have to only 1 leakage and how to deal with x2 and y2 in that case.
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt

2.12.1
# GPUs Available:  1


In [2]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data['x2'] = data['x2'].replace(np.nan, 0)
data['y2'] = data['y2'].replace(np.nan, 0)
# Not sure if 0 is good enough or try generating a random number

data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'tot_residual_flow', 
       'total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[['leak_1', 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[['leak_1', 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[['leak_1', 'leak_2']]

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

# scaler_coords2 = StandardScaler()
# y2_train = scaler_coords2.fit_transform(y2_train)
# y2_test = scaler_coords2.fit_transform(y2_test)
# y2_val = scaler_coords2.transform(y2_val)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

In [3]:
y_train = {
    "y1" : y1_train,
    "y2" : y2_train
}

y_val = {
    "y1" : y1_val,
    "y2" : y2_val
}

y_test = {
    "y1" : y1_test,
    "y2" : y2_test
}

losses = {
	"y1": "mse",
	"y2": 'binary_crossentropy'
    # "y2" : 'mse'
    }

metrics = {
    "y1": 'mae',
    "y2": 'mae'
    }



In [4]:
# def base_model(inputs):
#     # add the sequential layers here
#     x= Dense(128, activation='relu', kernel_initializer='he_uniform')(inputs)
#     # x= Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
#     return x

# def final_model(inputs):
#     x = base_model(inputs)
#     y1 = Dense(units=4, name='y1')(x)
#     y2 = Dense(units =2, name = 'y2')(x)
#     model = Model(inputs=inputs, outputs = [y1, y2])
    
#     return model

# inputs = tf.keras.layers.Input(shape=(11,))
# model = final_model(inputs)

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#             loss=losses, loss_weights=lossWeights,
#             metrics = metrics)

# history = model.fit(X_train, y_train, validation_data = (X_val, y_val), verbose = 1, epochs=100, shuffle = True)
# model_evaluate, y_pred = model_eval(model, X_test, y_test, X_train, y_train, X_val, y_val)
# coords = np.concatenate([y_pred[0], y_test['y1']], axis=1)
# presence = np.concatenate([y_pred[1], y_test['y2']], axis=1)

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import HyperModel, Hyperband

EPOCHS = 100
# Define the multi-task HyperModel
class MultiTaskHyperModel(HyperModel):

    def build(self, hp):
        inputs = keras.Input(shape=(10,))
        shared_layer = inputs
        for i in range(hp.Int('num_layers', 1, 5)):
            shared_layer = layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=128, step=32),
                activation=hp.Choice("activation", ["relu", "tanh", "elu"]),
                kernel_initializer='he_uniform'
            )(shared_layer)


        task_layer1 = shared_layer
        for j in range(hp.Int(f'task_{i}_num_layers', 0, 5)):
            task_layer1 = layers.Dense(units=hp.Choice(f'task_{i}_layer_{j}_neurons', values=[4, 8, 16]), activation='relu')(task_layer1)
        y1 = layers.Dense(1, name='y1')(task_layer1)

        task_layer2 = shared_layer
        for j in range(hp.Int(f'task_{i}_num_layers', 0, 5)):
            task_layer2 = layers.Dense(units=hp.Choice(f'task_{i}_layer_{j}_neurons', values=[4, 8, 16]), activation='relu')(task_layer2)
        y2 = layers.Dense(1, name='y2')(task_layer2)

        outputs = [y1, y2]

        loss1_weight = hp.Float("loss1_weight", min_value=1e-4, max_value=1, sampling="log")
        loss1_weight = hp.Float("loss2_weight", min_value=1e-4, max_value=1, sampling="log")

        # loss1_weight = hp.Choice('loss1_weight', values=[0.00001, 0.2, 0.4, 0.6, 0.8, 1.0])
        # loss2_weight = hp.Choice('loss2_weight', values=[0.00005, 0.2, 0.4, 0.6, 0.8, 1.0])

        lossWeights = {"y1": loss1_weight, "y2": loss1_weight}

        model = keras.Model(inputs=inputs, outputs=outputs)
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

        # model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        #             loss="mse",  metrics='mae')
        
        model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate),
                    loss=losses, loss_weights=lossWeights,
                    metrics = metrics)
        return model

# Create the multi-task HyperModel
multi_task_hypermodel = MultiTaskHyperModel()

# Define the Hyperband tuner
tuner = Hyperband(
    multi_task_hypermodel,
    objective='val_loss',
    max_epochs=EPOCHS,
    factor=5,
    directory="../../tensorflow_log_files/studienarbeit/",
    project_name='multi_task_tuning'
)

# Perform hyperparameter search
tuner.search(X_train, y_train, validation_data = (X_val, y_val), verbose = 1, epochs=EPOCHS, shuffle = True)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hps)

# Build the best model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the best model on the full dataset
best_model.fit(X_train, y_train, validation_data = (X_val, y_val), verbose = 1, epochs=EPOCHS, shuffle = True)

Trial 59 Complete [00h 00m 30s]
val_loss: 0.009099966358121974

Best val_loss So Far: 0.00012803264756660172
Total elapsed time: 00h 10m 12s
INFO:tensorflow:Oracle triggered exit
Best Hyperparameters: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7fc3507398e0>
Epoch 1/100
20/20 [==============================] - 3s 18ms/step - loss: 4.5419e-04 - y1_loss: 0.8746 - y2_loss: 2.7783 - y1_mae: 0.7373 - y2_mae: 1.8312 - val_loss: 5.2022e-04 - val_y1_loss: 0.7455 - val_y2_loss: 3.4384 - val_y1_mae: 0.6822 - val_y2_mae: 3.2940
Epoch 2/100
20/20 [==============================] - 0s 9ms/step - loss: 4.6442e-04 - y1_loss: 0.7066 - y2_loss: 3.0286 - y1_mae: 0.6805 - y2_mae: 4.4172 - val_loss: 5.7416e-04 - val_y1_loss: 0.6511 - val_y2_loss: 3.9667 - val_y1_mae: 0.6519 - val_y2_mae: 4.4355
Epoch 3/100
20/20 [==============================] - 0s 8ms/step - loss: 5.1100e-04 - y1_loss: 0.6898 - y2_loss: 3.4201 - y1_mae: 0.6734 - y2_mae: 5.3885 - val_loss: 6.8246e-04 -

In [6]:
best_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 dense_10 (Dense)               (None, 64)           704         ['input_2[0][0]']                
                                                                                                  
 dense_11 (Dense)               (None, 4)            260         ['dense_10[0][0]']               
                                                                                                  
 dense_12 (Dense)               (None, 4)            260         ['dense_10[0][0]']               
                                                                                            

In [7]:
y_predictions = best_model.predict(X_test)
results_train = best_model.evaluate(X_train, y_train, verbose=1)
results_val = best_model.evaluate(X_val, y_val, verbose=1)
results_test = best_model.evaluate(X_test, y_test, verbose=1)

5/5 [==============================] - 0s 3ms/step - loss: 3.4062e-04 - y1_loss: 0.6789 - y2_loss: 2.0607 - y1_mae: 0.6584 - y2_mae: 19.1397
